In [ ]:
USE {
    repositories {
        mavenCentral()
    }

    dependencies {
        implementation("org.apache.commons:commons-math3:3.6.1")
    }
}

In [ ]:
%use kandy
%use dataframe

In [ ]:
import kotlin.random.Random

// Prepare the measurement data 
fun getRandomMeasure(
    trueValue: Double,
    range: Double
): DoubleArray {

    val measureVal = Random.nextDouble(trueValue - range, trueValue + range)
    return  doubleArrayOf(measureVal)
}

In [149]:
import org.apache.commons.math3.filter.KalmanFilter
import org.apache.commons.math3.filter.DefaultProcessModel
import org.apache.commons.math3.filter.ProcessModel
import org.apache.commons.math3.filter.DefaultMeasurementModel
import org.apache.commons.math3.filter.MeasurementModel


val processModel: ProcessModel = DefaultProcessModel(
    arrayOf( //stateTransition: X
        doubleArrayOf(1.0)
    ),
    arrayOf( // control : U
        doubleArrayOf(1.0)
    ),
    arrayOf( // processNoise: Q
        doubleArrayOf(1.0)
    ),
    doubleArrayOf(50.0), // inital state estimate
    arrayOf( // initial covariance
        doubleArrayOf(100.0*100.0)
    ),
)

val measurementModel = DefaultMeasurementModel(
    arrayOf( //measMatrix: H
        doubleArrayOf(1.0)
    ),
    arrayOf( //measNoise : R
        doubleArrayOf(100.0*100)
    ),
)

val kalmanFilter = KalmanFilter(processModel, measurementModel)


In [150]:

val trueValue = 100.0
val range = 100.0


val estimate = mutableListOf<Double>()
val measurement = mutableListOf<Double>()
val covariances = mutableListOf<Double>()


repeat(100){ i ->
    
    kalmanFilter.predict()

    kalmanFilter.stateEstimation
    
    
    kalmanFilter.correct(
            getRandomMeasure(
                trueValue = trueValue ,
                range = range
            ).also {
                measurement.add(it[0])
            }
        )
    
    estimate.add(kalmanFilter.stateEstimation[0])
    covariances.add(kalmanFilter.errorCovariance[0][0])

}


val plotData = dataFrameOf(
    "data" to estimate +  measurement,
    "type" to  List(estimate.size) { "estimate" } + List(measurement.size) { "measurement" },
    "index" to List(estimate.size) { it } +  List(measurement.size) { it }
)


plotData.plot {
    line {
        x("index")
        y("data")
        color("type"){
            scale = categorical("measurement" to Color.BLUE, "estimate" to Color.RED )
        }
    }
    layout.size = 1000 to 450

}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="IoUmkC"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"data":[52.62264626856884,55.895134491231424,70.42371389777878,82.39827526054258,71.83048505127799,79.0154422184909,77.07783028645323,89.4233490524963,90.23259713409394,95.95916637162297,103.0998067768311,107.13152341849369,107.10887132498873,107.42638143176039,108.83741675301378,104.88450559505819,107.67575882386281,103.60357885967399,102.6728609662495,102.09173518390647,98.12380524068793,96.96252078601242,94.4543486570558,93.9278128663382,92.36605889425523,93.58574138902661,96.99860252510004,99.4467357395402,96.58922486480103,93.66818698123433,94.80521087566862,95.66780772504731,98.09720808064912,96.29288827379106,99.05767251551106,99.71006893611114,101.77925169568438,101.33640719296544,102.19722972467488,103.22978538082191,102.57980135086748,104.16471530950288,104.4429760349059,105.7681966934831,105.86550002031908,104.46044270334744,103.23499543145002,102.83440953399005,104.15134830682857,104.77118428310989,104.44508582670386,103.86337666879244,103.31240694558572,102.60213916665693,102.44948310592468,104.14510531693789,105.35266602312407,104.16384467749094,102.63779230291155,104.40213286530914,103.34530465080363,103.39168174470527,104.74925948328413,106.09197515040567,107.01884617751637,107.27868552365943,108.60181347264437,108.92943735976125,108.69458289595826,109.57328447227307,109.58135239557932,108.75822772633632,108.68880541911051,109.14199368815254,109.81711993375583,109.46958117552758,108.24948784507157,109.06509357828206,108.9617525173227,109.01298855709577,108.65235882972651,109.0985797525473,109.42986867283095,109.33849240650329,108.66667326160331,107.69376748678795,108.72000504110066,108.89572132504281,108.80952188020521,108.58367062501124,109.39160916520395,108.05811821705669,107.74517718925439,106.64741675133338,106.25304229531386,106.3052642689232,105.72355270362246,105.9367552560366,105.84056376418488,105.33449055683187,55.24503029873466,62.438475117757484,113.98912227113127,130.26062646055843,19.04958602854272,122.05991307215014,63.541612162970054,187.93637882320235,97.49284948370354,153.00535598417292,181.28743959456799,155.25169171878173,106.81623587215803,111.83956471242024,129.8296451637648,42.22290710959407,154.63485857835647,31.151557245157548,85.21566248854093,90.63324428677973,16.087199659983643,71.84622085363397,37.82747935085054,81.5429062461185,54.163202739936935,124.56182632511003,186.85018260685302,166.16539768341235,16.08378592349482,8.69982681488841,128.91382946466544,122.32408868276434,175.35435154142073,37.303047687887684,191.900585016052,122.19253556527919,174.89600044622674,85.30344395182348,134.10498930177246,142.3866059908757,77.37905564113386,166.9491502781292,115.69848341487064,160.4711900047891,109.96200996170758,44.161786134037165,49.65423842707055,84.9986023650443,163.83237201925235,133.3483796208949,89.15641459028221,76.14178606896964,76.63454932176488,67.67344675667609,94.82793385002894,190.0593080011467,167.42449495374265,42.191200449007106,21.987904611220642,198.9007199983757,45.99734592026379,105.94058932595615,180.29719844792942,181.7271033541446,159.8534303810752,122.2632752171538,185.77520540350565,128.2515769279889,94.69279105238537,162.51836083060462,110.07253438164655,58.13583353041505,104.37684195971988,137.5642662455401,152.56128836216286,87.26120069827851,29.573755633938227,162.12624497202845,102.18011312529352,112.40393951085603,84.58588468253643,139.11981986675409,131.89650263940297,103.09333512732675,62.39987330270866,40.19047763394188,180.44503700608203,121.26478361498158,102.69917383353663,92.46396844557066,167.444472108264,11.611852010809098,84.96545143324974,26.23

In [ ]:
val convarianceDf = dataFrameOf(
    "covariances" to covariances,
    "index" to List(covariances.size) { it } 
)


convarianceDf.plot {
    line {
        x("index")
        y("covariances")
    }
}